In [1]:
import pandas as pd
atrai_bike_data = pd.read_csv ('combined_data_21_01_2025.csv')

In [2]:
# Group by 'device_id' and count the number of rows for each device
device_counts = atrai_bike_data.groupby('device_id').size()

# Filter out device_ids with fewer than 10 data entries
valid_device_ids = device_counts[device_counts >= 10].index

# Filter the original DataFrame to keep only rows with valid device_ids
atrai_bike_data = atrai_bike_data[atrai_bike_data['device_id'].isin(valid_device_ids)]

In [3]:
import osmnx as ox

# Download road network for Münster, Germany
road_network_muenster = ox.graph_from_place("Münster, Germany", network_type='bike')

# Get the nodes and edges (roads) of the network
nodes, edges = ox.graph_to_gdfs(road_network_muenster)

# Filter out 'service' and 'residential' roads
edges_filtered = edges[~edges['highway'].isin(['primary', 'secondary', 'tertiary'])]

edges_filtered = edges_filtered.to_crs(epsg=32632)

# Apply simplification
#edges_filtered.loc[:, 'geometry'] = edges_filtered['geometry'].apply(lambda x: x.simplify(tolerance=0.5))

# Remove roads shorter than a specified length (in meters, for example)
#edges_filtered = edges_filtered[edges_filtered['geometry'].length > 10]

edges_filtered = edges_filtered.to_crs(epsg=4326)

In [ ]:
import folium
from shapely.geometry import Point, LineString
import geopandas as gpd
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import numpy as np
from sklearn.neighbors import BallTree
import branca.colormap as bc
from branca.colormap import linear
from branca.colormap import LinearColormap

# Function to find the nearest road segment (returns index of the segment)
def find_nearest_road_segment(point, road_network):
    distances = road_network.geometry.apply(lambda x: point.distance(x))
    nearest_idx = distances.idxmin()
    return nearest_idx  # Return the index instead of the name

# Define geographical bounds for Münster
min_lat, max_lat = 51.840, 52.061
min_lon, max_lon = 7.473, 7.775

# Filter bike data to include only points within Münster
filtered_data_MS = atrai_bike_data[
    (atrai_bike_data['lat'] >= min_lat) & (atrai_bike_data['lat'] <= max_lat) &
    (atrai_bike_data['lng'] >= min_lon) & (atrai_bike_data['lng'] <= max_lon)
].copy()

filtered_data_MS = filtered_data_MS[['createdAt', 'Speed', 'lat', 'lng', 'device_id', 'Standing']]
filtered_data_MS['createdAt'] = pd.to_datetime(filtered_data_MS['createdAt'])
filtered_data_MS = filtered_data_MS[filtered_data_MS['Standing'] < 0.75]

# Filter valid speeds (>= 0)
filtered_data_MS = filtered_data_MS[filtered_data_MS['Speed'] >= 0]

# Normalize speed using the 95th percentile
percentile_999 = filtered_data_MS['Speed'].quantile(0.999)
filtered_data_MS['Normalized_Speed'] = (filtered_data_MS['Speed'] / percentile_999).clip(upper=1)

# Step 1: Reproject edges to a projected CRS (e.g., EPSG:3857)
edges_projected = edges_filtered.to_crs("EPSG:3857")

# Step 2: Calculate centroids in the projected CRS
centroids = edges_projected.geometry.centroid

# Step 3: Reproject centroids back to EPSG:4326 for correct visualization
centroids = centroids.to_crs("EPSG:4326")

# Step 4: Reproject the edges back to EPSG:4326 after centroid calculation
edges_filtered = edges_projected.to_crs("EPSG:4326")

# Reindex edges to ensure alignment with BallTree
edges_filtered = edges_filtered.reset_index(drop=True)

# BallTree for nearest-neighbor search
road_coords = np.deg2rad(np.array([
    centroids.x.values,
    centroids.y.values
]).T)
tree = BallTree(road_coords, metric='haversine')

# Filter bike data and convert coordinates to radians
bike_coords = np.deg2rad(filtered_data_MS[['lng', 'lat']].values)
_, indices = tree.query(bike_coords, k=1)
filtered_data_MS['road_segment'] = indices.flatten()

# Aggregate data
segment_data = filtered_data_MS.groupby('road_segment').agg(
    avg_speed=('Normalized_Speed', 'mean'),
    points_in_segment=('road_segment', 'size')
).reset_index()

# Filter sparse segments
#segment_data = segment_data[segment_data['points_in_segment'] > 5]

# Set up color map
cmap = cm.get_cmap("plasma").reversed()

# Create the folium map
m_lines_dark = folium.Map(location=[51.95, 7.63], zoom_start=14,  tiles="Cartodb dark_matter", attr="Map tiles by CartoDB, under CC BY 3.0. Data by OpenStreetMap, under ODbL.")

segment_data['avg_speed_unnorm_kmh'] = (segment_data['avg_speed'] * percentile_999) * 3.6

# Add road segments to the map
for _, row in segment_data.iterrows():
    road_segment_idx = row['road_segment']
    road_segment = edges_filtered.loc[road_segment_idx]  # Use index to retrieve the segment
    
    if not road_segment.geometry.is_empty:
        line = road_segment.geometry
        color = mcolors.to_hex(cmap(row['avg_speed']))
        tooltip_text = f"Data Points: {row['points_in_segment']}<br>Avg Speed: {row['avg_speed_unnorm_kmh']:.2f} km/h"
            
        folium.PolyLine(
                locations=[(lat, lng) for lng, lat in line.coords],  # Convert LineString to (lat, lng)
                color=color,
                weight=4,
                tooltip=folium.Tooltip(tooltip_text)
            ).add_to(m_lines_dark)

# Generate a smooth gradient with many colors (high resolution)
num_colors = 256  # High number for smoothness
values = np.linspace(0, 1, num_colors)  # Cover the full range from 0.0 to 1.0

# Extract colors from the reversed colormap
colors = [mcolors.to_hex(cmap(v)) for v in values]

# Create a smooth branca colormap
colormap = LinearColormap(colors=colors, vmin=0, vmax=1)

# Create the custom HTML colorbar as a div element
html = """
    <div style="position: absolute; 
                bottom: 50px; 
                left: 50px; 
                width: 40px;  
                height: 200px; 
                background: linear-gradient(to top, """ + \
                ', '.join(colors) + \
                """); 
                border: 1px solid black; 
                opacity: 0.8; 
                padding: 5px; 
                z-index: 9999;">
        <!-- "Slow" label at the bottom with centered text and higher opacity -->
        <div style="position: absolute; 
                    bottom: 10px; 
                    left: 50%; 
                    transform: translateX(-50%); 
                    color: black; 
                    opacity: 1.0;  /* Full opacity for the label text */">
            <strong>Slow</strong>
        </div>
        <!-- "Fast" label at the top with centered text, higher opacity, and white color -->
        <div style="position: absolute; 
                    top: 10px; 
                    left: 50%; 
                    transform: translateX(-50%); 
                    color: white;  /* Changed text color to white */
                    opacity: 1.0;  /* Full opacity for the label text */">
            <strong>Fast</strong>
        </div>
    </div>
"""

# Add the HTML colorbar to the map using folium.Element
m_lines_dark.get_root().html.add_child(folium.Element(html))

m_lines_dark.save ("Speed_Münster_Dark.html")

C:\Users\sauer\AppData\Local\Temp\ipykernel_16800\2869666858.py:77: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap("plasma").reversed()


In [ ]:
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import numpy as np
from sklearn.neighbors import BallTree

# Function to find the nearest road segment (returns index of the segment)
def find_nearest_road_segment(point, road_network):
    distances = road_network.geometry.apply(lambda x: point.distance(x))
    nearest_idx = distances.idxmin()
    return nearest_idx  # Return the index instead of the name

# Define geographical bounds for Münster
min_lat, max_lat = 51.840, 52.061
min_lon, max_lon = 7.473, 7.775

# Filter bike data to include only points within Münster
filtered_data_MS = atrai_bike_data[
    (atrai_bike_data['lat'] >= min_lat) & (atrai_bike_data['lat'] <= max_lat) &
    (atrai_bike_data['lng'] >= min_lon) & (atrai_bike_data['lng'] <= max_lon)
].copy()

filtered_data_MS = filtered_data_MS[['createdAt', 'Speed', 'lat', 'lng', 'device_id', 'Standing']]
filtered_data_MS['createdAt'] = pd.to_datetime(filtered_data_MS['createdAt'])
#filtered_data_MS = filtered_data_MS[filtered_data_MS['Standing'] < 0.75]

# Filter valid speeds (>= 0)
filtered_data_MS = filtered_data_MS[filtered_data_MS['Speed'] >= 0]

# Normalize speed using the 95th percentile
percentile_999 = filtered_data_MS['Speed'].quantile(0.999)
filtered_data_MS['Normalized_Speed'] = (filtered_data_MS['Speed'] / percentile_999).clip(upper=1)

# Step 1: Reproject edges to a projected CRS (e.g., EPSG:3857)
edges_projected = edges_filtered.to_crs("EPSG:3857")

# Step 2: Calculate centroids in the projected CRS
centroids = edges_projected.geometry.centroid

# Step 3: Reproject centroids back to EPSG:4326 for correct visualization
centroids = centroids.to_crs("EPSG:4326")

# Step 4: Reproject the edges back to EPSG:4326 after centroid calculation
edges_filtered = edges_projected.to_crs("EPSG:4326")

# Reindex edges to ensure alignment with BallTree
edges_filtered = edges_filtered.reset_index(drop=True)

# BallTree for nearest-neighbor search
road_coords = np.deg2rad(np.array([
    centroids.x.values,
    centroids.y.values
]).T)
tree = BallTree(road_coords, metric='haversine')

# Filter bike data and convert coordinates to radians
bike_coords = np.deg2rad(filtered_data_MS[['lng', 'lat']].values)
_, indices = tree.query(bike_coords, k=1)
filtered_data_MS['road_segment'] = indices.flatten()

# Aggregate data
segment_data = filtered_data_MS.groupby('road_segment').agg(
    avg_speed=('Normalized_Speed', 'mean'),
    points_in_segment=('road_segment', 'size')
).reset_index()

# Filter sparse segments
#segment_data = segment_data[segment_data['points_in_segment'] >= 2]

cmap = cm.get_cmap("plasma").reversed()

# Create the folium map
m_lines = folium.Map(location=[51.95, 7.63], zoom_start=14)

segment_data['avg_speed_unnorm_kmh'] = (segment_data['avg_speed'] * percentile_999) * 3.6

# Add road segments to the map
for _, row in segment_data.iterrows():
    road_segment_idx = row['road_segment']
    road_segment = edges_filtered.loc[road_segment_idx]  # Use index to retrieve the segment
    
    if not road_segment.geometry.is_empty:
        line = road_segment.geometry
        color = mcolors.to_hex(cmap(row['avg_speed']))
        tooltip_text = f"Data Points: {row['points_in_segment']}<br>Avg Speed: {row['avg_speed_unnorm_kmh']:.2f} km/h"
            
        folium.PolyLine(
                locations=[(lat, lng) for lng, lat in line.coords],  # Convert LineString to (lat, lng)
                color=color,
                weight=4,
                tooltip=folium.Tooltip(tooltip_text)
            ).add_to(m_lines)

# Generate a smooth gradient with many colors (high resolution)
colors = [mcolors.to_hex(cmap(v)) for v in np.linspace(0, 1, 256)] 
min_speed = segment_data['avg_speed_unnorm_kmh'].min()
max_speed = segment_data['avg_speed_unnorm_kmh'].max()

# Define the speed values for the ticks (e.g., 25%, 50%, 75% between min and max speed)
tick_values = np.linspace(min_speed, max_speed, 5)  # Adjust the number of ticks as needed

# Generate HTML for the ticks
tick_marks = []
for tick in tick_values:
    position = ((tick - min_speed) / (max_speed - min_speed)) * 100  # Calculate position of tick
    tick_marks.append(f'<div style="position: absolute; left: {position}%; border-left: 2px solid black; height: 10px; margin-top: -5px; opacity: 0.6;"></div>')

# Generate HTML for the tick labels
tick_labels = []
for tick in tick_values:
    position = ((tick - min_speed) / (max_speed - min_speed)) * 100  # Calculate position of label
    tick_labels.append(f'<span style="position: absolute; left: {position}%; transform: translateX(-50%); font-size: 12px; margin-top: -10px; ">{tick:.1f}</span>')

# Combine the tick marks and labels
tick_marks_html = ''.join(tick_marks)
tick_labels_html = ''.join(tick_labels)

# Custom HTML legend (with full control over fonts and positions)
legend_html = f"""
    <div style="position: fixed; 
                top: 50px; right: 50px; width: 300px; height: 50px; 
                background-color: transparent; opacity: 0.9; padding: 10px; 
                border: none; z-index: 9999;">
        <div style="font-size: 18px; font-weight: bold; text-align: center;">
            <strong>Average Speed (km/h)</strong>
        </div>
        <div style="height: 20px; width: 100%; background: linear-gradient(to right, {', '.join(colors)});">
        </div>
        <!-- Add tick marks as physical ticks -->
        <div style="position: relative; height: 20px; width: 100%;">
            {tick_marks_html}
        </div>
        <!-- Add tick labels centered below the ticks -->
        <div style="font-size: 12px; display: flex; justify-content: space-between; padding-top: 5px; position: relative; width: 100%;">
            {tick_labels_html}
        </div>
    </div>
"""

# Add custom HTML legend directly to the map without using a popup or iframe
m_lines.get_root().html.add_child(folium.Element(legend_html))

m_lines.save ("Speed_Münster_OSM.html")

C:\Users\sauer\AppData\Local\Temp\ipykernel_16800\1516065119.py:76: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap("plasma").reversed()


In [ ]:
import folium
from shapely.geometry import Point, LineString
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
from sklearn.neighbors import BallTree
from branca.colormap import LinearColormap

# Function to find the nearest road segment (returns index of the segment)
def find_nearest_road_segment(point, road_network):
    distances = road_network.geometry.apply(lambda x: point.distance(x))
    nearest_idx = distances.idxmin()
    return nearest_idx  # Return the index instead of the name

# Define geographical bounds for Münster
min_lat, max_lat = 51.840, 52.061
min_lon, max_lon = 7.473, 7.775

# Filter bike data to include only points within Münster
filtered_data_MS_tf = atrai_bike_data[
    (atrai_bike_data['lat'] >= min_lat) & (atrai_bike_data['lat'] <= max_lat) &
    (atrai_bike_data['lng'] >= min_lon) & (atrai_bike_data['lng'] <= max_lon)
].copy()

filtered_data_MS_tf = filtered_data_MS_tf[['createdAt', 'Speed', 'lat', 'lng', 'device_id', 'Standing']]
filtered_data_MS_tf['createdAt'] = pd.to_datetime(filtered_data_MS_tf['createdAt'])

filtered_data_MS_tf = filtered_data_MS_tf.dropna(subset=['Standing'])

# Step 1: Sort the data by 'createdAt' to ensure chronological order
filtered_data_MS_tf = filtered_data_MS_tf.sort_values(by='createdAt')

# Step 2: Identify rides by checking time gaps greater than 1 minute
filtered_data_MS_tf['time_diff'] = filtered_data_MS_tf.groupby('device_id')['createdAt'].diff().dt.total_seconds() / 60  # Time difference in minutes
filtered_data_MS_tf['new_ride'] = filtered_data_MS_tf['time_diff'] > 1  # Mark where time gap > 1 minute (new ride)

# Step 3: Assign ride IDs for each 'device_id' based on time gaps
filtered_data_MS_tf['ride_id'] = filtered_data_MS_tf.groupby('device_id')['new_ride'].cumsum() + 1

# Step 4: Define the threshold for 'Standing' (e.g., Standing > 0.9)
standing_threshold = 0.9

# Step 5: Filter out consecutive start and end points where 'Standing' > threshold for each ride
def filter_start_end(group):
    # Remove consecutive start points with 'Standing' > threshold
    start_idx = group[group['Standing'] > standing_threshold].index
    if len(start_idx) > 0:
        group = group.drop(start_idx)
    
    # Remove consecutive end points with 'Standing' > threshold
    end_idx = group[group['Standing'] > standing_threshold].index
    if len(end_idx) > 0:
        group = group.drop(end_idx)
    
    return group

# Check the number of rows before and after filtering for 'Standing' > 0.9 at start/end
initial_rows = len(filtered_data_MS_tf)

# Step 6: Apply the filtering for each 'ride_id' (grouped by 'device_id' and 'ride_id')
filtered_data_MS_tf = filtered_data_MS_tf.groupby('ride_id').apply(lambda group: filter_start_end(group)).reset_index(drop=True)

# Step 7: Continue with the rest of your processing (normalization, aggregation, etc.)

# Filter valid speeds (>= 0)
filtered_data_MS_tf = filtered_data_MS_tf[filtered_data_MS_tf['Speed'] >= 0]

# Normalize speed using the 95th percentile
percentile_999_tf = filtered_data_MS_tf['Speed'].quantile(0.999)
filtered_data_MS_tf['Normalized_Speed'] = (filtered_data_MS_tf['Speed'] / percentile_999_tf).clip(upper=1)

# Calculate 'traffic_flow' combining 'Normalized_Speed' and 'Standing'
filtered_data_MS_tf['traffic_flow'] = (filtered_data_MS_tf['Normalized_Speed'] * (1 - (filtered_data_MS_tf['Standing'] ** 2)))

# Step 1: Reproject edges to a projected CRS (e.g., EPSG:3857)
edges_projected = edges_filtered.to_crs("EPSG:3857")

# Step 2: Calculate centroids in the projected CRS
centroids = edges_projected.geometry.centroid

# Step 3: Reproject centroids back to EPSG:4326 for correct visualization
centroids = centroids.to_crs("EPSG:4326")

# Step 4: Reproject the edges back to EPSG:4326 after centroid calculation
edges_filtered = edges_projected.to_crs("EPSG:4326")

# Reindex edges to ensure alignment with BallTree
edges_filtered = edges_filtered.reset_index(drop=True)

# BallTree for nearest-neighbor search
road_coords = np.deg2rad(np.array([
    centroids.x.values,
    centroids.y.values
]).T)
tree = BallTree(road_coords, metric='haversine')

# Filter bike data and convert coordinates to radians
bike_coords = np.deg2rad(filtered_data_MS_tf[['lng', 'lat']].values)
_, indices = tree.query(bike_coords, k=1)
filtered_data_MS_tf['road_segment'] = indices.flatten()

# Aggregate data based on 'traffic_flow'
segment_data = filtered_data_MS_tf.groupby('road_segment').agg(
    avg_traffic_flow=('traffic_flow', 'mean'),
    points_in_segment=('road_segment', 'size')
).reset_index()

# Create a segmented colormap with exact stops
cmap = cm.get_cmap('RdYlGn')
norm = mcolors.Normalize(vmin=0.3, vmax=0.7)

# Create the folium map
m_traffic_flow = folium.Map(location=[51.95, 7.63], zoom_start=14,  tiles="Cartodb dark_matter", attr="Map tiles by CartoDB, under CC BY 3.0. Data by OpenStreetMap, under ODbL.")

# Add road segments to the map based on traffic flow
for _, row in segment_data.iterrows():
    road_segment_idx = row['road_segment']
    road_segment = edges_filtered.loc[road_segment_idx]  # Use index to retrieve the segment
    
    if not road_segment.geometry.is_empty:
        line = road_segment.geometry
        color = mcolors.to_hex(cmap(norm(row['avg_traffic_flow'])))
        tooltip_text = f"Data Points: {row['points_in_segment']}"
        
        folium.PolyLine(
            locations=[(lat, lng) for lng, lat in line.coords],  # Convert LineString to (lat, lng)
            color=color,
            tooltip=folium.Tooltip(tooltip_text),
            weight=4,
        ).add_to(m_traffic_flow)
        
# Save the map
m_traffic_flow.save("Traffic_Flow_Münster_OSM.html")

C:\Users\sauer\AppData\Local\Temp\ipykernel_16800\2478144345.py:63: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_data_MS_tf = filtered_data_MS_tf.groupby('ride_id').apply(lambda group: filter_start_end(group)).reset_index(drop=True)
C:\Users\sauer\AppData\Local\Temp\ipykernel_16800\2478144345.py:117: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(custom_cmap)
